In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=2, threads_per_worker=2,dashboard_address=':8888')

client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:41519 Dashboard: http://127.0.0.1:8888/status,Cluster Workers: 2 Cores: 4 Memory: 35.01 GB


In [2]:
## Path for modules

import sys

import numpy as np
import xarray as xr

from netCDF4 import Dataset

sys.path.insert(0,"/mnt/meom/workdir/henelle/Notebooks/git/xscale")
import xscale
import xscale.spectral.fft as xfft

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy.ma as ma

sys.path.insert(0,"/mnt/meom/workdir/henelle/Notebooks/git/")
import myfunctions as mf

import matplotlib.cm as mplcm
import matplotlib.colors as colors
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import matplotlib as mpl
from datetime import date, datetime
from xhistogram.xarray import histogram

import pandas as pd

%matplotlib inline

### 1. Opening curl & strain files

In [3]:
season = 'JFM'

In [4]:
# With tide

dirvarT0     = '/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLBT02-S/1h/ACO/'
#curlfileT0   = dirvarT0 + 'eNATL60ACO-BLBT02_y2009m07-09_socurloverf10m_filt2T.nc' # JAS
#strainfileT0 = dirvarT0 + 'eNATL60ACO-BLBT02_y2009m07-09_sostrainoverf10m_filt2T.nc' # JAS

curlfileT0   = dirvarT0 + 'eNATL60ACO-BLBT02_y2010m01-03_socurloverf10m_filt2T.nc' # JFM
strainfileT0 = dirvarT0 + 'eNATL60ACO-BLBT02_y2010m01-03_sostrainoverf10m_filt2T.nc' # JFM

dscurlT0   = xr.open_mfdataset(curlfileT0,  combine='by_coords',parallel=True,chunks={'x':200,'y':200})
dsstrainT0 = xr.open_mfdataset(strainfileT0,combine='by_coords',parallel=True,chunks={'x':200,'y':200})

#curlT0   = dscurlT0.Curl_Tide_Filt # JAS
curlT0   = dscurlT0.curl_Tide_Filt # JFM
strainT0 = dsstrainT0.strain_Tide_Filt



In [5]:
# No tide

dirvar00     = '/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLB002-S/1h/ACO/'
#curlfile00   = dirvar00 + 'eNATL60ACO-BLB002_y2009m07-09_socurloverf10m_filt2T.nc' # JAS
#strainfile00 = dirvar00 + 'eNATL60ACO-BLB002_y2009m07-09_sostrainoverf10m_filt2T.nc' # JAS

curlfile00   = dirvar00 + 'eNATL60ACO-BLB002_y2010m01-03_socurloverf10m_filt2T.nc' # JFM
strainfile00 = dirvar00 + 'eNATL60ACO-BLB002_y2010m01-03_sostrainoverf10m_filt2T.nc' # JFM

dscurl00   = xr.open_mfdataset(curlfile00,  combine='by_coords',parallel=True,chunks={'x':200,'y':200})
dsstrain00 = xr.open_mfdataset(strainfile00,combine='by_coords',parallel=True,chunks={'x':200,'y':200})

#curl00   = dscurl00.Curl_noTide_Filt # JAS
curl00   = dscurl00.curl_noTide_Filt # JFM
strain00 = dsstrain00.strain_noTide_Filt

print(strain00.shape)

(2160, 1191, 781)


In [6]:
# Opening grid files

dirgrid = '/mnt/meom/workdir/alberta/eNATL60/eNATL60-I/'
gridfile = dirgrid+'mesh_hgr_eNATL60ACO_3.6.nc'
dsgrid = xr.open_dataset(gridfile,chunks={'x':200,'y':200})

lon = dsgrid.nav_lon
lat = dsgrid.nav_lat

In [7]:
lon.shape

(1191, 781)

### 2. Choosing region of interest

In [8]:
lonmin, lonmax, latmin, latmax = -31,-28,30.6,33.6 # region A3
#lonmin, lonmax, latmin, latmax = -31,-28,31.6,34.6 # region A1
#lonmin, lonmax, latmin, latmax = -31,-28,33,36 # region B
#lonmin, lonmax, latmin, latmax = -30,-27,30.6,33.6 # region A2
#lonmin, lonmax, latmin, latmax = -35.0,-32.5,35.4,39.0 # region C
#lonmin, lonmax, latmin, latmax = -34.0,-31.5,25.2,28.8 # region D

region = 'A3'

box = (lonmin, lonmax, latmin, latmax)

domain = (box[0]<lon)*(lon<box[1])*(box[2]<lat)*(lat<box[3])
where = np.where(domain)

In [9]:
jmin = where[0][0]
jmax = where[0][-1]
imin = where[1][0]
imax = where[1][-1]

In [10]:
print(imin,imax,jmin,jmax)

385 563 499 709


In [11]:
strainT0_filtbox = strainT0[:,jmin:jmax+1,imin:imax+1]
strain00_filtbox = strain00[:,jmin:jmax+1,imin:imax+1]
curlT0_filtbox   = curlT0  [:,jmin:jmax+1,imin:imax+1]
curl00_filtbox   = curl00  [:,jmin:jmax+1,imin:imax+1]

### 3. Defining bins and weights for histograms

In [12]:
sbin_width = 0.00625 # bin widths tested: 0.05 original, 0.025, 0.1, 0.0125
cbin_width = 0.0125  # bin widths tested: 0.1 original, 0.05, 0.2, 0.025

In [13]:
sbins = np.arange(0.0,1.0, sbin_width)
cbins = np.arange(-1.0,1.0, cbin_width)

In [14]:
number_sbins = (1.0 - 0.0) / sbin_width
number_cbins = (1.0 - (-1.0)) / cbin_width
print('Strain has '+str(int(number_sbins))+' bins.')
print('Curl has '+str(int(number_cbins))+' bins.')

Strain has 160 bins.
Curl has 160 bins.


In [15]:
strain00_filtbox.name ='strain00'
strainT0_filtbox.name ='strainT0'
curl00_filtbox.name   ='curl00'
curlT0_filtbox.name   ='curlT0'

In [16]:
### Hypothesis: We are neglecting the variation of the area since the regions chosen are small enough that the variation
### of the area is negligible in this case. 
hist_strain00_filtbox = histogram(strain00_filtbox,bins=[sbins])
hist_strainT0_filtbox = histogram(strainT0_filtbox,bins=[sbins])
hist_curl00_filtbox   = histogram(curl00_filtbox,bins=[cbins])
hist_curlT0_filtbox   = histogram(curlT0_filtbox,bins=[cbins])

In [17]:
#np.savez('hist_strain00_filtbox'+region+'.npz', hist_strain00_filtbox = hist_strain00_filtbox)
#np.savez('hist_strainT0_filtbox'+region+'.npz', hist_strainT0_filtbox = hist_strainT0_filtbox)
#np.savez('hist_curl00_filtbox'+region+'.npz', hist_curl00_filtbox = hist_curl00_filtbox)
#np.savez('hist_curlT0_filtbox'+region+'.npz', hist_curlT0_filtbox = hist_curlT0_filtbox)

In [18]:
np.savez('hist_strain00_bin'+region+season+'.npz', hist_strain00_bin = hist_strain00_filtbox.strain00_bin)
np.savez('hist_strainT0_bin'+region+season+'.npz', hist_strainT0_bin = hist_strainT0_filtbox.strainT0_bin)
np.savez('hist_curl00_bin'+region+season+'.npz', hist_curl00_bin = hist_curl00_filtbox.curl00_bin)
np.savez('hist_curlT0_bin'+region+season+'.npz', hist_curlT0_bin = hist_curlT0_filtbox.curlT0_bin)

### 4. Normalization n°2 with total count

In [19]:
# Convert tuple to int
nt = int(''.join(map(str, strainT0_filtbox[:,0,0].shape)))
ny = int(''.join(map(str, strainT0_filtbox[0,:,0].shape)))
nx = int(''.join(map(str, strainT0_filtbox[0,0,:].shape)))
print(nt,ny,nx)

2160 211 179


In [20]:
vol = nt * ny * nx
print(vol)

81581040


In [21]:
hist_strain00_norm2 = ( hist_strain00_filtbox / vol ) / sbin_width
hist_strainT0_norm2 = ( hist_strainT0_filtbox / vol ) / sbin_width

In [22]:
hist_curl00_norm2 = ( hist_curl00_filtbox / vol ) / cbin_width
hist_curlT0_norm2 = ( hist_curlT0_filtbox / vol ) / cbin_width

In [23]:
np.savez('hist_strain00_norm2'+region+season+'.npz', hist_strain00_norm2 = hist_strain00_norm2)
np.savez('hist_strainT0_norm2'+region+season+'.npz', hist_strainT0_norm2 = hist_strainT0_norm2)
np.savez('hist_curl00_norm2'+region+season+'.npz', hist_curl00_norm2 = hist_curl00_norm2)
np.savez('hist_curlT0_norm2'+region+season+'.npz', hist_curlT0_norm2 = hist_curlT0_norm2)